In [14]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [15]:
epochs = 20
BASE_DIR = '../'
batch_size = 32

In [16]:
def get_model():
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(299, 299, 3), output_shape=(299, 299, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(299, 299, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])

    return model

In [17]:
model = get_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_4 (Lambda)            (None, 299, 299, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 297, 297, 32)      896       
_________________________________________________________________
activation_16 (Activation)   (None, 297, 297, 32)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 146, 146, 32)      9248      
_________________________________________________________________
activation_17 (Activation)   (None, 146, 146, 32)      0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 73, 73, 32)        0         
__________

In [35]:
X_train = np.load('X_train_2.npy')
Y_train = np.load('Y_train_2.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('y_test.npy')

In [36]:
len(Y_test)

376

In [38]:
encoder = preprocessing.LabelEncoder()
encoder.fit(Y_train)
encoded_Y_train = encoder.transform(Y_train)
encoded_Y_test = encoder.transform(Y_test)

In [25]:
model = get_model()

# fits the model on batches
history = model.fit(
    X_train,
    encoded_Y_train,
    validation_split=0.2,
    epochs=epochs,
    shuffle=True,
    batch_size=batch_size)

model.save_weights('model1.h5')

Train on 320 samples, validate on 80 samples
Epoch 1/20
320/320 [==============================] - 69s 215ms/step - loss: 1.6195 - acc: 0.5062 - val_loss: 0.9140 - val_acc: 0.0000e+00
Epoch 2/20
320/320 [==============================] - 57s 179ms/step - loss: 0.6277 - acc: 0.6188 - val_loss: 1.6544 - val_acc: 0.0000e+00
Epoch 3/20
320/320 [==============================] - 67s 210ms/step - loss: 0.6381 - acc: 0.6219 - val_loss: 1.9532 - val_acc: 0.0000e+00
Epoch 4/20
320/320 [==============================] - 76s 237ms/step - loss: 0.5922 - acc: 0.6406 - val_loss: 2.5796 - val_acc: 0.0000e+00
Epoch 5/20
320/320 [==============================] - 75s 234ms/step - loss: 0.6801 - acc: 0.5781 - val_loss: 2.2679 - val_acc: 0.0000e+00
Epoch 6/20
320/320 [==============================] - 77s 240ms/step - loss: 0.5715 - acc: 0.6594 - val_loss: 4.7148 - val_acc: 0.0000e+00
Epoch 7/20
320/320 [==============================] - 75s 233ms/step - loss: 0.5729 - acc: 0.6562 - val_loss: 2.2090 - va

In [39]:
# test accuracy
Y_pred = np.rint(model.predict(X_test))

from sklearn.metrics import accuracy_score
print(accuracy_score(encoded_Y_test, Y_pred))

0.6968085106382979


In [40]:
# confusion matrix
# negative = 0 positive = 1
from sklearn.metrics import confusion_matrix

print(confusion_matrix(encoded_Y_test, Y_pred))
#Y_pred_unencoded = np.argmax(Y_pred, axis=0)
#Y_test_unencoded = np.argmax(Y_test, axis=0)
#print(confusion_matrix(Y_test_unencoded, Y_pred_unencoded))

[[245  81]
 [ 33  17]]
